In [1]:
import cv2
import numpy as np

import torch
from src.clip import clip
from PIL import Image

import os
import glob
from tqdm.auto import tqdm

from src.crop import video2npy, video_crop

c:\Users\ryuse\.conda\envs\XDVioDet\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device)

# 영상 저장 경로 (저장 시 안에 카테고리별로 영상 폴더를 만들어줘야 함)
root_folder_path = "D:/newGTAVideo"
# 피처 저장 경로
save_path = "D:/newGTACLIP"
os.makedirs(save_path, exist_ok=True)

category_folder = os.listdir(root_folder_path)
print(category_folder)

for category in tqdm(category_folder):
    print()
    print(category)
    os.makedirs(os.path.join(save_path, category), exist_ok=True)
    print()
    video_list = glob.glob(os.path.join(root_folder_path, os.path.join(category, "*.mp4")))
    
    # 각 카테고리 폴더의 영상들
    for video in tqdm(video_list):
        video_npy = video2npy(video, 16, 0) # (1799, 240, 320, 3)
        print()
        print("video: ", video_npy.shape)
        print()
        
        for i in range(10):
            corp_video_npy = video_crop(video_npy, i)                                   # (1799, 224, 224, 3)
            print("corp_video: ", corp_video_npy.shape)
                
            # 16번째 프레임마다 추출
            video_features = torch.zeros(0).to(device)
            with torch.no_grad():
                for j in range(0, corp_video_npy.shape[0]):
                    img = Image.fromarray(corp_video_npy[j])
                    img = preprocess(img).unsqueeze(0).to(device)
                    feature = model.encode_image(img)
                    video_features = torch.cat([video_features, feature], dim=0)
    
            video_features = video_features.detach().cpu().numpy()
            print("video feature: ", video_features.shape)
                
            np.save(os.path.join(os.path.join(save_path, category), video.split('\\')[-1].split('.')[0] + '__' + str(i) + '.npy'), video_features)